<a href="https://colab.research.google.com/github/sthanikan2000/CS3121-Project/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# Read the CSV file into a DataFrame
chatterbox = pd.read_csv('employees.csv')

In [3]:
# Replace "0000" with NaN in 'Year_of_Birth'
chatterbox['Year_of_Birth'] = chatterbox['Year_of_Birth'].replace("'0000'", '0000')
chatterbox['Year_of_Birth'] = chatterbox['Year_of_Birth'].astype('int64')
chatterbox['Year_of_Birth'] = chatterbox['Year_of_Birth'].replace(0, np.nan)

In [4]:
# Impute Date_resigned
mask = (chatterbox['Status'] == 'Inactive') & (chatterbox['Date_Resigned'].isin(['\\N', '0000-00-00'])) & (~chatterbox['Inactive_Date'].isin(['\\N', '0000-00-00']))
chatterbox.loc[mask, 'Date_Resigned'] = chatterbox.loc[mask, 'Inactive_Date']

# Replace '\\N' and '0000-00-00' with NaN
chatterbox['Date_Resigned'].replace(['\\N', '0000-00-00'], np.nan, inplace=True)
chatterbox['Inactive_Date'].replace(['\\N', '0000-00-00'], np.nan, inplace=True)
chatterbox['Reporting_emp_1'].replace(['\\N'], np.nan, inplace=True)
chatterbox['Reporting_emp_2'].replace(['\\N'], np.nan, inplace=True)


In [5]:
mask = (chatterbox["Gender"]=="Female")&(chatterbox["Title"]=="Mr")

# Since "Ms" is a title used for addressing a woman without specifying her marital status. It can be used for both married and unmarried women.
chatterbox.loc[mask, "Title"] = "Ms"

In [6]:
mask = (chatterbox["Gender"]=="Male")&(chatterbox["Title"].isin(["Ms","Miss"]))

# Since "Mr" is a title used for addressing a man without specifying his marital status. It can be used for both married and unmarried men.
chatterbox.loc[mask, "Title"] = "Mr"

In [7]:
mask = (chatterbox["Title"].isin(["Miss"]))&(chatterbox["Marital_Status"]=="Married")
chatterbox.loc[mask, "Marital_Status"] = "Single"

In [8]:
# Parse columns to datetime format to get all  date times in same format
for i in ["Date_Joined","Date_Resigned","Inactive_Date"]:
  chatterbox[i] = pd.to_datetime(chatterbox[i])

In [9]:
df = chatterbox[["Employee_No", 'Religion', "Title", "Gender", "Status", "Employment_Category", "Employment_Type", "Designation", "Marital_Status", "Year_of_Birth"]].copy()
df['Year_Joined'] = chatterbox['Date_Joined'].dt.year

# Label encoding the categorical nominal variables
le = LabelEncoder()

for i in ['Religion', "Title", "Gender", "Status", "Employment_Category", "Employment_Type", "Designation", "Marital_Status"]:
  if i == "Marital_Status":
    df[i + '_Encoded'] = le.fit_transform(df[i].astype(str))
    # Replace null values with a specific value, such as -1, to represent missing values
    df[i + '_Encoded'].fillna(-1, inplace=True)
  else:
    df[i + '_Encoded'] = le.fit_transform(df[i])
df = df.drop(['Religion', "Title", "Gender", "Status", "Employment_Category", "Employment_Type", "Designation","Marital_Status"], axis=1)


In [10]:
mask= chatterbox["Marital_Status"].isnull()
df.loc[mask, "Marital_Status_Encoded"] = -1

In [11]:
mask = (df["Marital_Status_Encoded"]== -1) & (df["Year_of_Birth"].isnull())
mask_temp=mask
df_copy=df.copy()
df=df.drop(df[mask].index)
mask.sum()

9

In [12]:
df_copy

,Employee_No,Year_of_Birth,Year_Joined,Religion_Encoded,Title_Encoded,Gender_Encoded,Status_Encoded,Employment_Category_Encoded,Employment_Type_Encoded,Designation_Encoded,Marital_Status_Encoded
0,347,1965.0,1993,0,1,1,0,2,1,31,0
1,348,1973.0,1995,0,1,1,0,2,1,31,0
2,349,1974.0,1988,0,1,1,1,2,1,0,0
3,351,1974.0,1999,1,2,0,1,2,1,85,0
4,352,1980.0,2001,0,1,1,0,2,1,108,0
...,...,...,...,...,...,...,...,...,...,...,...
992,2836,1993.0,2022,0,2,0,1,2,0,1,1
993,2890,1994.0,2022,0,1,1,0,2,0,89,1
994,2972,1997.0,2022,0,1,1,0,2,0,106,1
995,2973,1990.0,2022,0,1,1,0,2,0,89,1


In [13]:
df["Marital_Status_Encoded"].unique()

array([ 0,  1, -1])

In [14]:
df["Year_of_Birth"].unique()

array([1965., 1973., 1974., 1980., 1962., 1969., 1981., 1967., 1976.,
       1986., 1975., 1984., 1963., 1990., 1961., 1982., 1991., 1978.,
       1994., 1989., 1960., 1993., 1988., 1972., 1995., 1992., 1987.,
         nan, 1997., 1985., 1947., 1966., 1955., 1959., 1957., 1958.,
       1970., 2001., 1996., 1998., 1983., 1999., 1971., 1946., 1952.,
       1968., 1954., 1964., 2000., 1951., 2003., 1949., 1979., 1977.,
       1956., 1953., 2005., 2002., 1950., 2006.])

In [15]:
### Impute Year of Birth

# Step 1: Split the DataFrame into two sets
df_missing = df[df['Year_of_Birth'].isnull()]
df_not_missing = df[~df['Year_of_Birth'].isnull()]

# Step 2: Create the feature matrix X
X = df_not_missing[['Year_Joined', 'Employment_Category_Encoded', 'Employment_Type_Encoded', 'Gender_Encoded', 'Marital_Status_Encoded']]

# Step 3: Create the target array y
y = df_not_missing['Year_of_Birth']

# Step 4: Train a Random Forest Regressor model
model = RandomForestRegressor()
model.fit(X, y)

# Step 5: Predict missing values in df_missing
X_missing = df_missing[['Year_Joined', 'Employment_Category_Encoded', 'Employment_Type_Encoded', 'Gender_Encoded', 'Marital_Status_Encoded']]
predicted_values = model.predict(X_missing)

# Assign the predicted values to the missing entries in 'Year_of_Birth'
df.loc[df['Year_of_Birth'].isnull(), 'Year_of_Birth'] = predicted_values

In [16]:
df

,Employee_No,Year_of_Birth,Year_Joined,Religion_Encoded,Title_Encoded,Gender_Encoded,Status_Encoded,Employment_Category_Encoded,Employment_Type_Encoded,Designation_Encoded,Marital_Status_Encoded
0,347,1965.0,1993,0,1,1,0,2,1,31,0
1,348,1973.0,1995,0,1,1,0,2,1,31,0
2,349,1974.0,1988,0,1,1,1,2,1,0,0
3,351,1974.0,1999,1,2,0,1,2,1,85,0
4,352,1980.0,2001,0,1,1,0,2,1,108,0
...,...,...,...,...,...,...,...,...,...,...,...
992,2836,1993.0,2022,0,2,0,1,2,0,1,1
993,2890,1994.0,2022,0,1,1,0,2,0,89,1
994,2972,1997.0,2022,0,1,1,0,2,0,106,1
995,2973,1990.0,2022,0,1,1,0,2,0,89,1


In [17]:
df["Marital_Status_Encoded"].unique()

array([ 0,  1, -1])

In [18]:
# Step 1: Split the DataFrame into two sets
df_missing = df[df['Marital_Status_Encoded']==-1]
df_not_missing = df[~(df['Marital_Status_Encoded']==-1)]

# Step 2: Create the feature matrix X
X = df_not_missing[['Year_Joined', 'Employment_Category_Encoded', 'Employment_Type_Encoded', 'Gender_Encoded', 'Year_of_Birth', 'Title_Encoded']]

# Step 3: Create the target array y
y = df_not_missing['Marital_Status_Encoded']

# Step 4: Train a Random Forest Classifier model
model = RandomForestClassifier()
model.fit(X, y)

# Step 5: Predict missing values in df_missing
X_missing = df_missing[['Year_Joined', 'Employment_Category_Encoded', 'Employment_Type_Encoded', 'Gender_Encoded', 'Year_of_Birth', 'Title_Encoded']]
predicted_values = model.predict(X_missing)

# Assign the predicted values to the missing entries in 'Marital_Status_Encoded'
df.loc[df['Marital_Status_Encoded']==-1, 'Marital_Status_Encoded'] = predicted_values
df.loc[df['Marital_Status_Encoded']==0, 'Marital_Status'] = "Married"
df.loc[df['Marital_Status_Encoded']==1, 'Marital_Status'] = "Single"

In [19]:
df["Marital_Status"].unique()

array(['Married', 'Single'], dtype=object)

In [20]:
mask = chatterbox["Marital_Status"].isnull() & chatterbox["Year_of_Birth"].isnull()

chatterbox_missing=chatterbox.drop(chatterbox[~(mask)].index)
chatterbox=chatterbox.drop(chatterbox[mask].index)
chatterbox["Marital_Status"]=df["Marital_Status"]
chatterbox["Year_of_Birth"]=df["Year_of_Birth"]

chatterbox = pd.concat([chatterbox, chatterbox_missing])
chatterbox["Marital_Status"] =chatterbox["Marital_Status"].fillna(chatterbox["Marital_Status"].mode()[0])
chatterbox["Year_of_Birth"] =chatterbox["Year_of_Birth"].fillna(chatterbox["Year_of_Birth"].mean())

chatterbox['Year_of_Birth'] = chatterbox['Year_of_Birth'].astype(int)

# Order the DataFrame by Employee_No
chatterbox = chatterbox.sort_values(by='Employee_No')


In [21]:
x=chatterbox["Marital_Status"].mode()[0]
x

'Married'

In [22]:
chatterbox.isnull().sum()

Employee_No              0
Employee_Code            0
Name                     0
Title                    0
Gender                   0
Religion_ID              0
Marital_Status           0
Designation_ID           0
Date_Joined              0
Date_Resigned          233
Status                   0
Inactive_Date          233
Reporting_emp_1        936
Reporting_emp_2        997
Employment_Category      0
Employment_Type          0
Religion                 0
Designation              0
Year_of_Birth            0
dtype: int64

In [23]:
chatterbox

,Employee_No,Employee_Code,Name,Title,Gender,Religion_ID,Marital_Status,Designation_ID,Date_Joined,Date_Resigned,Status,Inactive_Date,Reporting_emp_1,Reporting_emp_2,Employment_Category,Employment_Type,Religion,Designation,Year_of_Birth
0,347,6,Myrl Bechtelar MD,Mr,Male,1,Married,24,1993-12-08,NaT,Active,NaT,NaN,NaN,Staff,Permanant,Buddhist,Driver,1965
1,348,33,Francis Mraz,Mr,Male,1,Married,24,1995-03-14,NaT,Active,NaT,NaN,NaN,Staff,Permanant,Buddhist,Driver,1973
2,349,53,Mr Emmet Blick Jr,Mr,Male,1,Married,23,1988-01-27,2021-06-28,Inactive,2021-06-28,NaN,NaN,Staff,Permanant,Buddhist,Account Clerk,1974
3,351,77,Mr Nelson Braun PhD,Ms,Female,4,Married,40,1999-10-01,2022-01-31,Inactive,2022-01-31,NaN,NaN,Staff,Permanant,Catholic,Purchasing Officer,1974
4,352,88,Maximillian Hand,Mr,Male,1,Married,7,2001-01-26,NaT,Active,NaT,NaN,NaN,Staff,Permanant,Buddhist,Store Keeper,1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,2836,5434,Janis Borer Jr,Ms,Female,1,Single,17,2022-11-01,2022-12-22,Inactive,2022-12-22,NaN,NaN,Staff,Contarct Basis,Buddhist,Account Executive,1993
993,2890,5479,Delphia Haley,Mr,Male,1,Single,8,2022-11-21,NaT,Active,NaT,NaN,NaN,Staff,Contarct Basis,Buddhist,Quantity Surveyor,1994
994,2972,5480,Lance Stanton,Mr,Male,1,Single,201,2022-12-05,NaT,Active,NaT,NaN,NaN,Staff,Contarct Basis,Buddhist,Software Developer,1997
995,2973,5481,Chauncey Reilly,Mr,Male,1,Single,8,2022-12-08,NaT,Active,NaT,NaN,NaN,Staff,Contarct Basis,Buddhist,Quantity Surveyor,1990


In [25]:
chatterbox.duplicated().sum()

0

In [26]:
chatterbox.to_csv('employee_preprocess_200635E.csv', index=False)
